In [3]:
import numpy as np

In [2]:
import pandas as pd
import functools
import math
import fractions


In [10]:
#Поиск максимального модуля в массиве
def AbsMax(A):
    return np.max(np.array([np.max(A), -np.min(A)]))

In [ ]:
#Спектральный радиус матрицы
def SpecRad (A):
        return AbsMax(np.linalg.eig(A)[0])

In [8]:
#Замена вектора на вектор из целых
def integer_collinear_vector(numbers, max_denominator=1000000):
    t = np.array([])
    for i in range(len(numbers)):
        if numbers[i] != 0:
            a = np.array([numbers[i]])
            t = np.concatenate((t, a))
    norma = np.min(t)
    numbers = [num / norma for num in numbers]
    fracts = [fractions.Fraction(f).limit_denominator(max_denominator) for f in numbers]
    denoms = [f.denominator for f in fracts]
    denoms_gcd = functools.reduce(math.gcd, denoms)
    denoms_lcm = functools.reduce(lambda a, b: a*b, [d // denoms_gcd for d in denoms]) * denoms_gcd
    result_integers = [f.numerator * (denoms_lcm // f.denominator) for f in fracts]
    return result_integers

Метод Якоби

In [12]:
def yakob(A,X,F,N):
    Y = np.zeros(N)
    for i in range(N):
        for j in range(N):
            if j == i:
                continue
            Y[i] += -(X[j]*(A[i][j]))
        Y[i] += F[i]
        Y[i] = Y[i]/(A[i][i])
    X = Y
    return X

In [23]:
#Размер матриц NxN
N = 3
A = np.array([[18.0,6.0,0.0],[6.0,6.0,-7.0],[0.0,-7.0,18.0]])
F = np.array([-6.0,-15.0,30.0])
X = np.array([0.0,0.0,1.0])
for i in range(10):
    X = yakob(A,X,F,N)
    print(X)

[-0.33333333 -1.33333333  1.66666667]
[ 0.11111111 -0.22222222  1.14814815]
[-0.25925926 -1.27160494  1.58024691]
[ 0.09053498 -0.39711934  1.17215364]
[-0.20096022 -1.22302241  1.51223137]
[ 0.0743408  -0.53476985  1.19104684]
[-0.15507672 -1.18478615  1.45870061]
[ 0.06159538 -0.6431059   1.2059165 ]
[-0.1189647  -1.1546928   1.41656993]
[ 0.05156427 -0.72837039  1.21761946]


Метод Зейделя

In [20]:
def zeid(A,X,F,N):
    for i in range(N):
        X[i] = 0
        for j in range(N):
            if j == i:
                continue
            X[i] += -(X[j]*(A[i][j]))
        X[i] += F[i]
        X[i] = X[i]/(A[i][i])
    return X

In [24]:
#Размер матриц NxN
N = 3
A = np.array([[18,0,6],[0,18,-7],[6,-7,6]])
F = np.array([-6,30,-15])
X = np.array([0.0,1.0,0.0])
for i in range(10):
    X = zeid(A,X,F,N)
    print(X)

[-0.33333333  1.66666667 -0.22222222]
[-0.25925926  1.58024691 -0.39711934]
[-0.20096022  1.51223137 -0.53476985]
[-0.15507672  1.45870061 -0.6431059 ]
[-0.1189647   1.41656993 -0.72837039]
[-0.0905432   1.38341152 -0.79547669]
[-0.06817444  1.35731462 -0.84829184]
[-0.05056939  1.33677539 -0.88985932]
[-0.03671356  1.32061026 -0.92257446]
[-0.02580851  1.30788771 -0.9483225 ]


Метод верхрней релаксации
<br>
$1<w<2$
<br>
$U_i^{s+1} =  U_i^s (1-w)  - \frac{w}{a_{ii}} (\sum_{j=1}^{i-1}a_{ij} u_j^{s+1} +\sum_{j=i+1}^{n}a_{ij} u_j^{s} - f_i)$

In [26]:
#Оптимальное значение для метода релаксации (если выполнен критерий)
def OptRel(A):
    D = np.diag(A)
    Q = np.identity(len(A))
    G = np.identity(len(A))
    for i in range(len(A)):
        Q[i][i] = 1/D[i]
        G[i][i] = D[i]
    R = -Q@(A - G)
    return (2/(1 + (1 - (SpecRad(R))**2)**0.5))
#Метод релаксации
def relax(A,X,F,N,w):
    for i in range(N):
        X[i] = X[i]*(1-w)*A[i][i]/w
        for j in range(N):
            if j == i:
                continue
            X[i] += -(X[j]*(A[i][j]))
        X[i] += F[i]
        X[i] = X[i]*w/(A[i][i])
    return X

In [27]:
#Размер матриц NxN
N = 3
A = np.array([[18,0,6],[0,18,-7],[6,-7,6]])
F = np.array([-6,30,-15])
X = np.array([0.0,1.0,0.0])
w = 1.368
for i in range(20):
    X = relax(A,X,F,N,w)
    print(X)

[-0.456    1.912    0.25536]
[-0.40463616  1.71223552 -0.22770232]
[-0.20326163  1.52875969 -0.61824316]
[-0.09928084  1.38851107 -0.84060666]
[-0.03614801  1.32182518 -0.95157327]
[-0.00878012  1.28733135 -1.003229  ]
[ 0.00470351  1.27254424 -1.02626552]
[ 0.01024619  1.26573046 -1.03624526]
[ 0.01275724  1.26292871 -1.04047942]
[ 0.01376395  1.26170718 -1.042248  ]
[ 0.01419995  1.26121582 -1.04297782]
[ 0.0143723   1.26100838 -1.0432761 ]
[ 0.0144449   1.26092603 -1.04339707]
[ 0.01447334  1.26089198 -1.04344581]
[ 0.0144851   1.26087858 -1.04346534]
[ 0.01448968  1.26087312 -1.04347314]
[ 0.01449155  1.26087098 -1.04347623]
[ 0.01449227  1.26087012 -1.04347746]
[ 0.01449257  1.26086979 -1.04347795]
[ 0.01449268  1.26086965 -1.04347814]


Метод простых итераций:

$u + \tau A u = \tau f + u$
<br>
$u^{s+1} = u^s  + \tau(f - A u^s)$
<br>
$r^s := f - A u^s$
<br>
$u^{s+1} = u^s  + \tau r^s$
<br>
$\delta^s = u^s - u^{*}$

In [30]:
def k(u,t,A,f):
    u = u+(f-A@u)*t
    return  u

Вычисление максимального по модулю собственного числа

$\lambda^{s+1} = \frac{(u^s,Au^s)}{(u^s,u^s)}$
<br>
$O(\frac{|\lambda_{beforemax}|}{|\lambda_{max}|})$

In [17]:
def scal(x,y):
    m = x@(y.transpose())
    return m

In [21]:
u = np.array([1, 0])
t = 0.5
A = np.array([[5.0,-2.0],[-2,2]])
f = np.array([4, 5,6])
for j in range(5):
    print("Шаг №:", j+1)
    print("Вектор:",u)
    print("(u,Au):",scal(u,A@u))
    print("(u,u):",scal(u,u))
    print("lambda",scal(u,A@u)/scal(u,u))
    u = A@u

Шаг №: 1
Вектор: [1 0]
(u,Au): 5.0
(u,u): 1
lambda 5.0
Шаг №: 2
Вектор: [ 5. -2.]
(u,Au): 173.0
(u,u): 29.0
lambda 5.9655172413793105
Шаг №: 3
Вектор: [ 29. -14.]
(u,Au): 6221.0
(u,u): 1037.0
lambda 5.999035679845709
Шаг №: 4
Вектор: [173. -86.]
(u,Au): 223949.0
(u,u): 37325.0
lambda 5.999973208305425
Шаг №: 5
Вектор: [1037. -518.]
(u,Au): 8062157.0
(u,u): 1343693.0
lambda 5.9999992557823845


$tg (2\alpha) = \frac{2a_{ij}}{a_{ii} - a{jj}}$

Вычисление решения при матрице $A_{(m,n)}$, $rk(A) = n$:
<br>
$Au = f$
<br>
$A^{T} A x = A^{T} f$
<br>
$(A^{T}A)_{(n,n)}$
<br>
Тоесть это уже система с квадратной матрицей, которую мы решать умеем

Невязка:
<br>
$F(u) = \sum_i(\sum_j a_{ij}u_j - f_i)^2$

In [30]:
def nevya(u,A,f,n):
    s = 0
    for i in range(n):
        k = 0
        for j in range(n):
            k += (A[i][j] * u[j] - f[i])
        s += k ** 2
    return s

In [31]:
n = 2
A = np.array([[1,1],[2,-1],[1,3],[3,1]])
f = np.array([3,0.2,7.0,5.0])
u = np.linalg.inv((A.transpose()@A))@A.transpose()@f
print("Вектор:",u)
print("Невязка:",nevya(u,A,f,n))

Вектор: [1.03741935 1.96774194]
Невязка: 9.054851196670137
